In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from sklearn.model_selection import train_test_split



from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, Ridge, ElasticNet

from sklearn.svm import SVR

from sklearn.neighbors import KNeighborsRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor

from xgboost import XGBRegressor

from lightgbm import LGBMRegressor



from sklearn.metrics import mean_squared_error

import warnings

warnings.filterwarnings('ignore')

sns.set()

In [3]:
# FUNCTIONS

def check_missing(df, drop_list):

  for col in df.columns:

    if df[col].isnull().sum() > 0 :

      if df[col].isnull().sum() > 0.5 * len(df):

        drop_list.append(col)

      print(f"{col} - {df[col].isnull().sum()} - {df[col].dtype}")

In [4]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [5]:
droping = []

check_missing(df, droping)

print(droping)

LotFrontage - 259 - float64
Alley - 1369 - object
MasVnrType - 872 - object
MasVnrArea - 8 - float64
BsmtQual - 37 - object
BsmtCond - 37 - object
BsmtExposure - 38 - object
BsmtFinType1 - 37 - object
BsmtFinType2 - 38 - object
Electrical - 1 - object
FireplaceQu - 690 - object
GarageType - 81 - object
GarageYrBlt - 81 - float64
GarageFinish - 81 - object
GarageQual - 81 - object
GarageCond - 81 - object
PoolQC - 1453 - object
Fence - 1179 - object
MiscFeature - 1406 - object
['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature']


In [6]:
df.drop(droping, axis=1, inplace=True)

df.drop("Id", axis=1, inplace=True)

df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [7]:
check_missing(df, droping)

LotFrontage - 259 - float64
MasVnrArea - 8 - float64
BsmtQual - 37 - object
BsmtCond - 37 - object
BsmtExposure - 38 - object
BsmtFinType1 - 37 - object
BsmtFinType2 - 38 - object
Electrical - 1 - object
FireplaceQu - 690 - object
GarageType - 81 - object
GarageYrBlt - 81 - float64
GarageFinish - 81 - object
GarageQual - 81 - object
GarageCond - 81 - object


In [8]:
for col in df.columns:

  if df[col].dtype == 'object':

    df[col].fillna(df[col].mode()[0], inplace=True)

  else:

    df[col].fillna(df[col].median(), inplace=True)

check_missing(df, droping)

In [9]:
encoder = LabelEncoder()

for col in df.columns:

  if df[col].dtype == 'object':

    df[col] = encoder.fit_transform(df[col])

df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,3,65.0,8450,1,3,3,0,4,0,...,0,0,0,0,0,2,2008,8,4,208500
1,20,3,80.0,9600,1,3,3,0,2,0,...,0,0,0,0,0,5,2007,8,4,181500
2,60,3,68.0,11250,1,0,3,0,4,0,...,0,0,0,0,0,9,2008,8,4,223500
3,70,3,60.0,9550,1,0,3,0,0,0,...,272,0,0,0,0,2,2006,8,0,140000
4,60,3,84.0,14260,1,0,3,0,2,0,...,0,0,0,0,0,12,2008,8,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,1,3,3,0,4,0,...,0,0,0,0,0,8,2007,8,4,175000
1456,20,3,85.0,13175,1,3,3,0,4,0,...,0,0,0,0,0,2,2010,8,4,210000
1457,70,3,66.0,9042,1,3,3,0,4,0,...,0,0,0,0,2500,5,2010,8,4,266500
1458,20,3,68.0,9717,1,3,3,0,4,0,...,112,0,0,0,0,4,2010,8,4,142125


In [10]:
scaler = MinMaxScaler()



X = df.drop('SalePrice', axis=1)

y = df['SalePrice']

X = scaler.fit_transform(X)

y_df = y.to_frame()

y = scaler.fit_transform(y_df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [11]:
models = []

model_1 = LinearRegression()

model_2 = SGDRegressor()

model_3 = Lasso()

model_4 = Ridge()

model_5 = ElasticNet()

model_6 = SVR()

model_7 = KNeighborsRegressor()

model_8 = DecisionTreeRegressor()

model_9 = RandomForestRegressor()

model_10 = BaggingRegressor()

model_11 = ExtraTreesRegressor()

model_12 = AdaBoostRegressor()

model_13 = XGBRegressor(verbose=0)

model_14 = LGBMRegressor(verbose=0)

for i in range(1, 15):

  exec(f"model_{i}.fit(X_train, y_train)")

  exec(f"models.append(model_{i})")

models_names = ['LinearRegression', 'SGDRegressor', 'Lasso', 'Ridge', 'ElasticNet', 'SVR', 'KNeighborsRegressor', 'DecisionTreeRegressor', 'RandomForestRegressor', 'BaggingRegressor', 'ExtraTreesRegressor', 'AdaBoostRegressor', 'XGBRegressor', 'LGBMRegressor']

In [12]:
def ML_models(x_train, x_valid, y_train, y_valid, models, models_names):

  # Calculate predictions and squared errors for each model:

  squared_errors = []

  for model in models:

      y_pred = model.predict(x_valid)

      mse = mean_squared_error(y_valid, y_pred)

      squared_errors.append(f'{mse * 100:.2f}%')  # Format as percentage



  # Calculate train and test scores:

  train_score = [model.score(x_train, y_train) for model in models]

  valid_score = [model.score(x_valid, y_valid) for model in models]



  # Difference between training and testing ratio

  ratio = []

  for train, valid in zip(train_score, valid_score):

      result = train - valid

      ratio.append(f'{result * 100:.2f}%')



  # Measure model state:6

  rate = []

  for train, valid in zip(train_score, valid_score):

      if train <= 0.65 and valid <= 0.65:

          rate.append('bad')

      elif train > valid * 1.10:

          rate.append('overfite')

      elif train > 0.65 and train < 0.80 and valid > 0.65 and valid < 0.80:

          rate.append('middle')

      elif train >= 0.80 and valid >= 0.80 and train < 1.00 and valid < 1.00:

          rate.append('good')

      elif train >= 0.80 and valid < 0.80:

          rate.append('high train, low test')

      else:

          rate.append('unknown')



  # Create DataFrame

  model_score = pd.DataFrame({

      'Model': models_names,

      'Train score': [f'{round(score * 100, 2)}%' for score in train_score],

      'Valid score': [f'{round(score * 100, 2)}%' for score in valid_score],

      'Ratio difference': ratio,

      'Evaluate model': rate,

      'Squared error': squared_errors

  })

  return model_score

In [13]:
result = ML_models(X_train, X_test, y_train, y_test, models, models_names)

result

,Model,Train score,Valid score,Ratio difference,Evaluate model,Squared error
0,LinearRegression,84.56%,84.68%,-0.11%,good,0.23%
1,SGDRegressor,78.04%,78.23%,-0.18%,middle,0.32%
2,Lasso,0.0%,-0.09%,0.09%,bad,1.48%
3,Ridge,84.44%,84.58%,-0.13%,good,0.23%
4,ElasticNet,0.0%,-0.09%,0.09%,bad,1.48%
5,SVR,78.54%,74.16%,4.39%,middle,0.38%
6,KNeighborsRegressor,80.85%,74.96%,5.89%,"high train, low test",0.37%
7,DecisionTreeRegressor,100.0%,78.75%,21.25%,overfite,0.31%
8,RandomForestRegressor,97.92%,89.15%,8.77%,good,0.16%
9,BaggingRegressor,96.64%,89.58%,7.05%,good,0.15%


In [14]:
# prompt: grid search for XGBRegressor



from sklearn.model_selection import GridSearchCV



# Define the parameter grid for XGBRegressor

param_grid = {

    'n_estimators': [100, 200, 300],

    'learning_rate': [0.01, 0.1, 0.2],

    'max_depth': [3, 5, 7],

    'subsample': [0.8, 1.0],

    'colsample_bytree': [0.8, 1.0]

}



# Initialize XGBRegressor

xgb_model = model_13



# Perform grid search

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)



# Print the best parameters and best score

print("Best parameters found: ", grid_search.best_params_)

print("Lowest RMSE found: ", np.sqrt(np.abs(grid_search.best_score_)))



# Evaluate the best model

best_xgb_model = grid_search.best_estimator_

y_pred = best_xgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error of best model: {mse}")

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.8; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.8; total time=   0.2s
[CV] END 

# **TEST AND SUBMISSION**

In [15]:
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [16]:
droping = []

check_missing(df_test, droping)

droping.remove('FireplaceQu')

print(droping)

MSZoning - 4 - object
LotFrontage - 227 - float64
Alley - 1352 - object
Utilities - 2 - object
Exterior1st - 1 - object
Exterior2nd - 1 - object
MasVnrType - 894 - object
MasVnrArea - 15 - float64
BsmtQual - 44 - object
BsmtCond - 45 - object
BsmtExposure - 44 - object
BsmtFinType1 - 42 - object
BsmtFinSF1 - 1 - float64
BsmtFinType2 - 42 - object
BsmtFinSF2 - 1 - float64
BsmtUnfSF - 1 - float64
TotalBsmtSF - 1 - float64
BsmtFullBath - 2 - float64
BsmtHalfBath - 2 - float64
KitchenQual - 1 - object
Functional - 2 - object
FireplaceQu - 730 - object
GarageType - 76 - object
GarageYrBlt - 78 - float64
GarageFinish - 78 - object
GarageCars - 1 - float64
GarageArea - 1 - float64
GarageQual - 78 - object
GarageCond - 78 - object
PoolQC - 1456 - object
Fence - 1169 - object
MiscFeature - 1408 - object
SaleType - 1 - object
['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature']


In [17]:
df_test.drop(droping, axis=1, inplace=True)

df_test.drop("Id", axis=1, inplace=True)

df_test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,120,0,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,34,0,0,0,0,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,...,82,0,0,144,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,32,0,0,0,0,700,7,2006,WD,Normal


In [18]:
for col in df_test.columns:

  if df_test[col].dtype == 'object':

    df_test[col].fillna(df_test[col].mode()[0], inplace=True)

  else:

    df_test[col].fillna(df_test[col].median(), inplace=True)

check_missing(df_test, droping)

In [19]:
encoder = LabelEncoder()

for col in df_test.columns:

  if df_test[col].dtype == 'object':

    df_test[col] = encoder.fit_transform(df_test[col])

df_test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,2,80.0,11622,1,3,3,0,4,0,...,0,0,0,120,0,0,6,2010,8,4
1,20,3,81.0,14267,1,0,3,0,0,0,...,36,0,0,0,0,12500,6,2010,8,4
2,60,3,74.0,13830,1,0,3,0,4,0,...,34,0,0,0,0,0,3,2010,8,4
3,60,3,78.0,9978,1,0,3,0,4,0,...,36,0,0,0,0,0,6,2010,8,4
4,120,3,43.0,5005,1,0,1,0,4,0,...,82,0,0,144,0,0,1,2010,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,4,21.0,1936,1,3,3,0,4,0,...,0,0,0,0,0,0,6,2006,8,4
1455,160,4,21.0,1894,1,3,3,0,4,0,...,24,0,0,0,0,0,4,2006,8,0
1456,20,3,160.0,20000,1,3,3,0,4,0,...,0,0,0,0,0,0,9,2006,8,0
1457,85,3,62.0,10441,1,3,3,0,4,0,...,32,0,0,0,0,700,7,2006,8,4


In [20]:
y_pred = best_xgb_model.predict(df_test)

y_pred

array([0.5185506 , 0.4359921 , 0.6435375 , ..., 0.5270386 , 0.39211276,
       0.62170434], dtype=float32)

In [21]:
# Reshape y_pred to 2D array

y_pred_reshaped = y_pred.reshape(-1, 1)



# Transform the reshaped predictions

scaled_data = scaler.transform(y_pred_reshaped)



# Inverse transform the reshaped predictions

y_pred_rescaler = scaler.inverse_transform(y_pred_reshaped)



# Output the final predictions

y_pred_rescaler

array([[408308.28],
       [348857.9 ],
       [498311.38],
       ...,
       [414420.47],
       [317260.4 ],
       [482589.28]], dtype=float32)

In [22]:
submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

submission['SalePrice'] = y_pred_rescaler

submission.to_csv('submission.csv', index=False)